In [19]:
import sqlite3
from pathlib import Path
import sknw
from skimage.morphology import skeletonize
import networkx as nx
import numpy as np

In [20]:
# path to databases
print("For which experiment would you like to reduce the data?")
experiment = int(input())
if experiment == 1:
    db_path = Path('E:/HumanA/Data/DataBase/HumanA_Exp1.db')
    #db_path = Path('E:/HumanA/Data/HumanA_Exp1_WorkingData.db')
elif experiment == 2:
    db_path = Path('E:/HumanA/Data/DataBase/HumanA_Exp2.db')
    #db_path = Path('E:/HumanA/Data/HumanA_Exp2_WorkingData.db')

if not db_path or not db_path.exists():
    db_path = ':memory:'

For which experiment would you like to reduce the data?
2


In [21]:
# connect to database
connection=sqlite3.connect(db_path)
cr=connection.cursor()

In [22]:
def create_graph_coor_db_table():
    # create a table that stores the graph coordinates information
    sql_instruction = """
    CREATE TABLE IF NOT EXISTS "graph_coordinates"(
    "Id" INTEGER NOT NULL UNIQUE,
    "GraphId" INTEGER NOT NULL,
    "graph_element_type" TEXT,
    "nodeNr" NUMERIC,
    "nodeCentroid_x" NUMERIC,
    "nodeCentroid_z" NUMERIC,
    "nodeRadius" NUMERIC,
    "edgeStart" NUMERIC,
    "edgeEnd" NUMERIC,
    "X_Coor" NUMERIC,
    "Z_Coor" NUMERIC,
    PRIMARY KEY ("Id" AUTOINCREMENT)
    FOREIGN KEY(GraphId) REFERENCES graph_information(Id)
    );
    """
    cr.execute(sql_instruction)
    connection.commit()

In [23]:
def create_node_neighbours_table():
    # create a table that stores the graph coordinates information
    sql_instruction = """
    CREATE TABLE IF NOT EXISTS "node_neighbours"(
    "FirstNode" INTEGER NOT NULL,
    "SecondNode" INTEGER NOT NULL,
    PRIMARY KEY ("FirstNode", "SecondNode" )
    FOREIGN KEY(FirstNode) REFERENCES graph_coordinates(nodeNr)
	FOREIGN KEY(SecondNode) REFERENCES graph_coordinates(nodeNr)
    );
    """
    cr.execute(sql_instruction)
    connection.commit()

In [24]:
# get the id of the corresponding graph
def get_graph_id(graphname):
    # get the id from the graph
    cr.execute(f"""SELECT Id FROM graph_information WHERE Graph_Name = '{graphname}' """)
    graphId = cr.fetchone()
    graphId = graphId[0]
    return graphId

In [25]:
# check if the node is already in the db
def node_in_graph_coor(node):

    sql_instruction = """
    SELECT nodeNr FROM graph_coordinates
    """
    
    cr.execute(sql_instruction)
    content = cr.fetchall()
    if (node,) in content:
        return True
    else:
        return False

In [26]:
# check if the edge is already saved in the database
def edge_in_graph_coor(edge_start, edge_end):

    sql_instruction = """
    SELECT edgeStart,edgeEnd FROM graph_coordinates
    """

    cr.execute(sql_instruction)
    content = cr.fetchall()
    if (edge_start,edge_end) in content:
        return True
    else:
        return False

In [27]:
def NodeOrNeighbour_in_db(node, neighbour):
    sql_instruction = """
    SELECT FirstNode, SecondNode FROM node_neighbours
    """
    
    cr.execute(sql_instruction)
    content = cr.fetchall()
    if (node,neighbour) in content or (neighbour,node) in content:
        return True
    else:
        return False

In [28]:
def add_neighbours_to_DB(graph):
    nodes = graph.nodes()
    for node in nodes:
        neighbours = list(graph.neighbors(node))
        for neighbour in neighbours:
            if not NodeOrNeighbour_in_db(node, neighbour):
                values = str((node, neighbour))
                sql_instruction = f"""INSERT INTO node_neighbours
                                    (FirstNode, SecondNode)
                                    VALUES {values}
                                    ;"""
                cr.execute(sql_instruction)
                connection.commit()

In [29]:
# add the nodes coordinates and centroid coordinates to db
def add_nodes_to_db(graph):
    nodes = graph.nodes()
    graphId = get_graph_id('Exploration_Graph')
    graph_element_type = 'Node'
    # go through all nodes of the graph
    for node in nodes:
        if not node_in_graph_coor(node):
            pts = nodes[node]['pts']
            radius = nodes[node]['r']
            # save coordinates to db
            values = (str((graphId, graph_element_type, node, nodes[node]['o'][0], nodes[node]['o'][1], radius, point[0], 
                           point[1])) for point in pts)

            sql_instruction = f"""INSERT INTO graph_coordinates
                                (GraphId, graph_element_type, nodeNr, nodeCentroid_x, nodeCentroid_z,nodeRadius, X_Coor, Z_Coor)
                                VALUES {','.join(values)}
                                ;"""
            cr.execute(sql_instruction)
            
            
            
            connection.commit()

In [30]:
# add the edges coordinates to the db
def add_edges_to_db(graph):
    graphId = get_graph_id('Exploration_Graph')
    graph_element_type = 'Edge'
    
    # go through all edges in the graph
    for start,end in graph.edges():
        if not edge_in_graph_coor(start,end) and graph[start][end] != {}:
            # get the position/ coordinates for the edge
            pts = graph[start][end]['pts']
            # if there is more than one coordinate save all coordinates
            if len(pts) > 2:
                values = (str((graphId, graph_element_type, start, end, point[0], point[1])) for point in pts)
                sql_instruction = f"""INSERT INTO graph_coordinates
                                    (GraphId, graph_element_type, edgeStart,edgeEnd, X_Coor, Z_Coor)
                                    VALUES {','.join(values)}
                                    ;"""
                cr.execute(sql_instruction)
                connection.commit()
            # else save only the two coordinates
            else:
                values = str((graphId, graph_element_type, start, end, pts[0], pts[1]))

                sql_instruction = f"""INSERT INTO graph_coordinates
                                    (GraphId, graph_element_type, edgeStart,edgeEnd, X_Coor, Z_Coor)
                                    VALUES {values}
                                    ;"""
                cr.execute(sql_instruction)
            connection.commit()

In [31]:
# get nodes and radius from db
def get_Nodes_and_Radius():
    sql_instruction = """ SELECT Nodes, Radius FROM nodes_radius"""
    cr.execute(sql_instruction)
    nodes_data = cr.fetchall()

    nodes = list(list(zip(*nodes_data))[0])
    r = list(list(zip(*nodes_data))[1])
    radius_list = dict(zip(*(nodes,r)))

    return radius_list

In [32]:
# get mask from database
def get_graph_mask():
    sql_instruction = """ SELECT Row, Column,Value FROM mask_information"""
    cr.execute(sql_instruction)
    mask_values = cr.fetchall()

    # recreate mask from database
    last_row = 0
    mask_values_current_row = []
    mask_values_complete = []
    for i in mask_values:
        if i[0] == last_row:
            mask_values_current_row.append(i[2])
        else:
            mask_values_complete.append(mask_values_current_row)
            mask_values_current_row = [i[2]]
        last_row = i[0]

    mask = np.asarray(mask_values_complete)
    return mask

In [33]:
# rebuild the graph from the databank
def build_graph(mask, radius_list):
    skeleton = skeletonize(mask)

    temp_graph = sknw.build_sknw(skeleton)
    nx.set_node_attributes(temp_graph, radius_list, "r")
    temp_graph.add_edge(96,104)
    graph = temp_graph.copy()
    return graph

In [34]:
create_graph_coor_db_table()
create_node_neighbours_table()

radius_list = get_Nodes_and_Radius()
mask = get_graph_mask()
graph = build_graph(mask, radius_list)
start_point = [76,60]

add_nodes_to_db(graph)
add_edges_to_db(graph)
add_neighbours_to_DB(graph)

In [35]:
##nodes = graph.nodes()
#max_count_neighbours = 0
##count_neighbours = None
#for node in graph.nodes():
#    neighbours = list(graph.neighbors(node))
#    count_neighbours = 0
#    for neighbour in neighbours:
#        count_neighbours += 1
#        if count_neighbours > max_count_neighbours:
#            print("Node with most neighbours so far: " + str(node))
#            max_count_neighbours = count_neighbours
#            print(max_count_neighbours)

In [36]:
connection.close()